### Registration and masks

**Learning outcomes:**
- How to apply registration to an image and its corresponding mask using ants.

In [1]:
%matplotlib inline

import os
from help_functions import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\Windows\Desktop\GBM_Segmentation


#### Raw image

In [3]:
raw_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'brain-lesion_T1w.nii')
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')

explore_3D_array(arr=raw_img_ants.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [4]:
mask_img_path = os.path.join(BASE_DIR,  'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'brain-lesion_T1w_mask.nii')
mask_img_ants = ants.image_read(mask_img_path, reorient='IAL')

explore_3D_array_with_mask_contour(
    arr=raw_img_ants.numpy(),
    mask=mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

#### Template image

In [5]:
template_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
template_img_ants = ants.image_read(template_img_path, reorient='IAL')

explore_3D_array(arr = template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [6]:
print('\t\tRAW IMG')
print(raw_img_ants)

print('\t\tTEMPLATE IMG')
print(template_img_ants)


		RAW IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]

		TEMPLATE IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]



#### Registration

In [7]:
transformation = ants.registration(
    fixed=template_img_ants,
    moving=raw_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [00000163AFF59990,00000163AFF59CB0,1] -m mattes[00000163AFF59990,00000163AFF59CB0,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[00000163AFF59990,00000163AFF59CB0,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\Windows\AppData\Local\Temp\tmp4v5kom0r,00000163AFF59A20,00000163AFF59B30] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [8]:
print(transformation)

{'warpedmovout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'fwdtransforms': ['C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp4v5kom0r1Warp.nii.gz', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp4v5kom0r0GenericAffine.mat'], 'invtransforms': ['C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp4v5kom0r0GenericAffine.mat', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp4v5kom0r1InverseWarp.nii.gz']}


In [9]:
registered_img_ants = transformation['warpedmovout']

explore_3D_array(arr=registered_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

#### Move raw mask from native space.

In [10]:
registered_mask_img_ants = ants.apply_transforms(
    moving=mask_img_ants,
    fixed=transformation['warpedmovout'],
    transformlist=transformation['fwdtransforms'],
    verbose=True
)

['-d', '3', '-i', '00000163B8EBC2B0', '-o', '00000163B8EBC2A0', '-r', '00000163B8EBC2F0', '-n', 'linear', '-t', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp4v5kom0r1Warp.nii.gz', '-t', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp4v5kom0r0GenericAffine.mat']


In [11]:
explore_3D_array_with_mask_contour(
    arr=registered_img_ants.numpy(),
    mask=registered_mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…